<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 54.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK',
       'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART',
       'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND',
       'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    55
1    10
Name: InPortfolio, dtype: int64

In [6]:
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0.0    7
1.0    3
Name: CCIFlag, dtype: int64

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [8]:
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [9]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI
Date,,,,,,,,,,
2023-01-02 00:00:00+05:30,18197.0,^NSEI,5.57,0.49,0.40,18812.0,15294.0,82.53,17.47,48.0
2023-01-03 00:00:00+05:30,18233.0,^NSEI,5.73,0.49,0.37,18812.0,15294.0,83.53,16.47,49.0
2023-01-04 00:00:00+05:30,18043.0,^NSEI,4.59,0.48,0.32,18812.0,15294.0,78.14,21.86,43.0
2023-01-05 00:00:00+05:30,17992.0,^NSEI,4.28,0.45,0.27,18812.0,15294.0,76.70,23.30,41.0
2023-01-06 00:00:00+05:30,17859.0,^NSEI,3.49,0.42,0.23,18812.0,15294.0,72.92,27.08,38.0


In [11]:
# MACD Uptrend
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)

In [12]:
# portfolio stocks
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
2,2023-01-06 00:00:00+05:30,473.0,ICICIPRULI.NS,-8.33,0.00,-0.85,615.0,440.0,18.86,81.14,60.0,636.0,60.9,79.50,34.0
4,2023-01-06 00:00:00+05:30,702.0,MOTILALOFS.NS,-8.30,-0.91,0.00,960.0,654.0,15.82,84.18,55.0,930.0,19.7,9.63,32.0
5,2023-01-06 00:00:00+05:30,382.0,TATAMOTORS.NS,-10.54,-0.23,-0.24,525.0,372.0,6.54,93.46,35.0,491.0,19.2,1000.00,29.0
8,2023-01-06 00:00:00+05:30,1269.0,SBILIFE.NS,6.02,0.75,0.08,1332.0,1023.0,79.63,20.37,55.0,1607.0,62.1,1000.00,27.0
9,2023-01-06 00:00:00+05:30,3687.0,NAUKRI.NS,-8.85,-0.77,-0.15,5667.0,3363.0,14.06,85.94,31.0,4677.0,78.8,16.00,27.0
13,2023-01-06 00:00:00+05:30,1448.0,INFY.NS,-4.89,-0.92,0.00,1898.0,1351.0,17.82,82.18,31.0,1733.0,23.0,26.60,20.0
21,2023-01-06 00:00:00+05:30,986.0,TECHM.NS,-9.36,-1.93,-0.29,1684.0,935.0,6.83,93.17,36.0,1131.0,18.7,18.20,15.0
23,2023-01-06 00:00:00+05:30,1072.0,MUTHOOTFIN.NS,-2.79,-1.27,0.37,1516.0,956.0,20.76,79.24,47.0,1217.0,12.5,11.50,14.0
34,2023-01-06 00:00:00+05:30,612.0,HDFCLIFE.NS,9.81,0.54,1.06,672.0,502.0,64.50,35.50,71.0,684.0,89.3,89.30,12.0
53,2023-01-06 00:00:00+05:30,3435.0,DIVISLAB.NS,-7.96,-1.15,-0.56,4595.0,3266.0,12.71,87.29,52.0,3560.0,45.2,30.50,4.0


In [13]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 4


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
1,2023-01-06 00:00:00+05:30,568.0,BERGEPAINT.NS,-9.49,-0.64,-0.33,787.0,558.0,4.54,95.46,31.0,782.0,72.9,58.4,38.0
6,2023-01-06 00:00:00+05:30,498.0,JUBLFOOD.NS,-10.43,-0.18,-1.65,785.0,463.0,10.87,89.13,34.0,636.0,87.9,69.4,28.0
13,2023-01-06 00:00:00+05:30,1448.0,INFY.NS,-4.89,-0.92,0.00,1898.0,1351.0,17.82,82.18,31.0,1733.0,23.0,26.6,20.0
18,2023-01-06 00:00:00+05:30,2538.0,TITAN.NS,4.19,0.08,-0.38,2770.0,1929.0,72.43,27.57,46.0,2980.0,78.0,71.1,17.0


In [14]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

2     15.887850
4     14.953271
5     13.551402
8     12.616822
9     12.616822
13     9.345794
21     7.009346
23     6.542056
34     5.607477
53     1.869159
Name: Upside%, dtype: float64

In [15]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 14


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
0,2023-01-06 00:00:00+05:30,451.0,GODREJIND.NS,-0.38,-0.66,0.23,644.0,405.0,19.37,80.63,56.0,638.0,39.5,19.6,41.0
1,2023-01-06 00:00:00+05:30,568.0,BERGEPAINT.NS,-9.49,-0.64,-0.33,787.0,558.0,4.54,95.46,31.0,782.0,72.9,58.4,38.0
3,2023-01-06 00:00:00+05:30,5980.0,BAJFINANCE.NS,-10.33,-0.12,-1.22,7824.0,5265.0,27.96,72.04,25.0,7966.0,50.8,36.5,33.0
6,2023-01-06 00:00:00+05:30,498.0,JUBLFOOD.NS,-10.43,-0.18,-1.65,785.0,463.0,10.87,89.13,34.0,636.0,87.9,69.4,28.0
7,2023-01-06 00:00:00+05:30,238.0,ZEEL.NS,-5.26,0.00,-1.18,325.0,206.0,26.72,73.28,36.0,303.0,24.6,28.2,27.0
10,2023-01-06 00:00:00+05:30,870.0,ICICIBANK.NS,6.24,1.34,-0.33,953.0,650.0,72.64,27.36,35.0,1094.0,24.2,20.5,26.0
11,2023-01-06 00:00:00+05:30,1625.0,BATAINDIA.NS,-9.02,-0.45,-1.06,2020.0,1587.0,8.74,91.26,37.0,2042.0,62.3,67.5,26.0
12,2023-01-06 00:00:00+05:30,77.0,IBREALEST.NS,-3.50,-1.25,0.00,174.0,60.0,15.09,84.91,41.0,95.0,11.3,1000.0,23.0
14,2023-01-06 00:00:00+05:30,3251.0,EICHERMOT.NS,5.23,1.49,-1.29,3850.0,2241.0,62.75,37.25,46.0,3916.0,40.1,38.1,20.0
15,2023-01-06 00:00:00+05:30,1182.0,INDUSINDBK.NS,13.64,1.54,0.59,1264.0,771.0,83.35,16.65,45.0,1402.0,22.8,15.1,19.0


In [16]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 85]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 6


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
37,2023-01-06 00:00:00+05:30,382.0,WIPRO.NS,-12.59,-2.29,0.25,704.0,375.0,2.13,97.87,39.0,425.0,19.0,18.6,11.0
1,2023-01-06 00:00:00+05:30,568.0,BERGEPAINT.NS,-9.49,-0.64,-0.33,787.0,558.0,4.54,95.46,31.0,782.0,72.9,58.4,38.0
11,2023-01-06 00:00:00+05:30,1625.0,BATAINDIA.NS,-9.02,-0.45,-1.06,2020.0,1587.0,8.74,91.26,37.0,2042.0,62.3,67.5,26.0
6,2023-01-06 00:00:00+05:30,498.0,JUBLFOOD.NS,-10.43,-0.18,-1.65,785.0,463.0,10.87,89.13,34.0,636.0,87.9,69.4,28.0
27,2023-01-06 00:00:00+05:30,205.0,TATAPOWER.NS,-9.91,-0.44,-0.45,287.0,195.0,11.30,88.70,39.0,232.0,20.4,25.8,13.0
41,2023-01-06 00:00:00+05:30,4198.0,LTIM.NS,-9.63,-1.98,-1.01,7153.0,3796.0,11.96,88.04,35.0,4675.0,26.6,48.5,11.0


In [17]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] < 10]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 5


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
62,2023-01-06 00:00:00+05:30,293.0,INDIANB.NS,48.21,4.04,2.53,300.0,127.0,96.21,3.79,56.0,284.0,7.4,8.40,-3.0
55,2023-01-06 00:00:00+05:30,116.0,TATASTEEL.NS,31.42,3.41,1.83,119.0,44.0,95.53,4.47,58.0,118.0,6.0,4.91,2.0
26,2023-01-06 00:00:00+05:30,940.0,AXISBANK.NS,21.75,1.55,0.66,962.0,625.0,93.44,6.56,56.0,1060.0,31.3,15.70,13.0
32,2023-01-06 00:00:00+05:30,135.0,FEDERALBNK.NS,21.26,1.80,0.00,139.0,82.0,92.28,7.72,50.0,152.0,11.0,11.40,13.0
58,2023-01-06 00:00:00+05:30,183.0,BANKBARODA.NS,42.05,3.10,2.33,193.0,84.0,91.06,8.94,56.0,183.0,13.5,9.62,0.0


In [18]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
50,2023-01-06 00:00:00+05:30,3212.0,TCS.NS,-2.38,-0.46,0.36,3965.0,2974.0,23.97,76.03,40.0,3385.0,28.1,29.7,5.0
27,2023-01-06 00:00:00+05:30,205.0,TATAPOWER.NS,-9.91,-0.44,-0.45,287.0,195.0,11.30,88.70,39.0,232.0,20.4,25.8,13.0
40,2023-01-06 00:00:00+05:30,3842.0,DMART.NS,-4.70,-0.17,-0.72,4731.0,3231.0,40.77,59.23,39.0,4282.0,138.7,108.0,11.0
17,2023-01-06 00:00:00+05:30,1430.0,BAJAJFINSV.NS,-5.65,-0.07,-0.98,1836.0,1093.0,45.40,54.60,29.0,1691.0,39.2,41.7,18.0
11,2023-01-06 00:00:00+05:30,1625.0,BATAINDIA.NS,-9.02,-0.45,-1.06,2020.0,1587.0,8.74,91.26,37.0,2042.0,62.3,67.5,26.0
3,2023-01-06 00:00:00+05:30,5980.0,BAJFINANCE.NS,-10.33,-0.12,-1.22,7824.0,5265.0,27.96,72.04,25.0,7966.0,50.8,36.5,33.0
22,2023-01-06 00:00:00+05:30,1937.0,DEEPAKNTR.NS,-5.90,-0.24,-1.31,2650.0,1713.0,23.86,76.14,34.0,2231.0,28.4,28.7,15.0
6,2023-01-06 00:00:00+05:30,498.0,JUBLFOOD.NS,-10.43,-0.18,-1.65,785.0,463.0,10.87,89.13,34.0,636.0,87.9,69.4,28.0


In [19]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 6


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
0,2023-01-06 00:00:00+05:30,451.0,GODREJIND.NS,-0.38,-0.66,0.23,644.0,405.0,19.37,80.63,56.0,638.0,39.5,19.60,41.0
36,2023-01-06 00:00:00+05:30,81.0,NATIONALUM.NS,-2.05,-2.41,2.60,128.0,67.0,23.44,76.56,58.0,90.0,10.3,5.88,11.0
37,2023-01-06 00:00:00+05:30,382.0,WIPRO.NS,-12.59,-2.29,0.25,704.0,375.0,2.13,97.87,39.0,425.0,19.0,18.60,11.0
38,2023-01-06 00:00:00+05:30,463.0,HINDALCO.NS,7.34,-1.16,1.56,626.0,314.0,47.64,52.36,51.0,514.0,10.4,7.50,11.0
45,2023-01-06 00:00:00+05:30,1033.0,HCLTECH.NS,2.77,-0.50,0.09,1291.0,865.0,39.39,60.61,42.0,1117.0,16.3,20.30,8.0
50,2023-01-06 00:00:00+05:30,3212.0,TCS.NS,-2.38,-0.46,0.36,3965.0,2974.0,23.97,76.03,40.0,3385.0,28.1,29.70,5.0


In [20]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 5


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
44,2023-01-06 00:00:00+05:30,2020.0,INDIGO.NS,9.13,0.59,1.36,2265.0,1549.0,65.78,34.22,55.0,2176.0,30.7,1000.0,8.0
24,2023-01-06 00:00:00+05:30,1594.0,HDFCBANK.NS,9.43,0.75,1.00,1662.0,1281.0,82.26,17.74,43.0,1813.0,26.0,21.3,14.0
29,2023-01-06 00:00:00+05:30,2596.0,HDFC.NS,9.03,0.88,0.99,2729.0,2053.0,80.33,19.67,41.0,2940.0,23.0,19.6,13.0
47,2023-01-06 00:00:00+05:30,6982.0,ULTRACEMCO.NS,8.81,0.78,0.89,7825.0,5148.0,68.52,31.48,48.0,7420.0,33.7,30.2,6.0
30,2023-01-06 00:00:00+05:30,553.0,DABUR.NS,1.25,0.37,0.53,604.0,480.0,58.75,41.25,39.0,625.0,53.6,54.7,13.0


In [21]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
36,2023-01-06 00:00:00+05:30,81.0,NATIONALUM.NS,-2.05,-2.41,2.60,128.0,67.0,23.44,76.56,58.0,90.0,10.3,5.88,11.0
38,2023-01-06 00:00:00+05:30,463.0,HINDALCO.NS,7.34,-1.16,1.56,626.0,314.0,47.64,52.36,51.0,514.0,10.4,7.50,11.0
50,2023-01-06 00:00:00+05:30,3212.0,TCS.NS,-2.38,-0.46,0.36,3965.0,2974.0,23.97,76.03,40.0,3385.0,28.1,29.70,5.0
37,2023-01-06 00:00:00+05:30,382.0,WIPRO.NS,-12.59,-2.29,0.25,704.0,375.0,2.13,97.87,39.0,425.0,19.0,18.60,11.0
0,2023-01-06 00:00:00+05:30,451.0,GODREJIND.NS,-0.38,-0.66,0.23,644.0,405.0,19.37,80.63,56.0,638.0,39.5,19.60,41.0
45,2023-01-06 00:00:00+05:30,1033.0,HCLTECH.NS,2.77,-0.50,0.09,1291.0,865.0,39.39,60.61,42.0,1117.0,16.3,20.30,8.0
12,2023-01-06 00:00:00+05:30,77.0,IBREALEST.NS,-3.50,-1.25,0.00,174.0,60.0,15.09,84.91,41.0,95.0,11.3,1000.00,23.0
59,2023-01-06 00:00:00+05:30,86.0,SAIL.NS,6.05,-1.23,0.00,108.0,63.0,50.89,49.11,55.0,85.0,6.9,7.48,-1.0
1,2023-01-06 00:00:00+05:30,568.0,BERGEPAINT.NS,-9.49,-0.64,-0.33,787.0,558.0,4.54,95.46,31.0,782.0,72.9,58.40,38.0
41,2023-01-06 00:00:00+05:30,4198.0,LTIM.NS,-9.63,-1.98,-1.01,7153.0,3796.0,11.96,88.04,35.0,4675.0,26.6,48.50,11.0
